In [1]:
#VAE code from PyTorch Tutorial

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid

In [4]:
epochs = 10

parser = argparse.ArgumentParser(description='VAE MNIST Example')


device = torch.device("cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} 
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=512, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=512, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 10) # dimension of latent sapce = 10
        self.fc22 = nn.Linear(400, 10)
        self.fc3 = nn.Linear(10, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            '''
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

            '''
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    #print('====> Test set loss: {:.4f}'.format(test_loss))

if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 10).to(device)
            sample = model.decode(sample).cpu()
            #save_image(sample.view(64, 1, 28, 28),'cur_result/sample_' + str(epoch) + '.png')

====> Epoch: 1 Average loss: 225.2875
====> Epoch: 2 Average loss: 149.6425
====> Epoch: 3 Average loss: 133.8037
====> Epoch: 4 Average loss: 127.3975
====> Epoch: 5 Average loss: 123.8701
====> Epoch: 6 Average loss: 121.5272
====> Epoch: 7 Average loss: 119.8532
====> Epoch: 8 Average loss: 118.5398
====> Epoch: 9 Average loss: 117.4358
====> Epoch: 10 Average loss: 116.6239


In [5]:
torch.save(model, './model')

/usr/local/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [6]:
model = torch.load('./model')

In [13]:
import matplotlib.pyplot as plt
import numpy as np


temp = torch.randn(64, 10).to(device)
temp = model.decode(temp).cpu()

with torch.no_grad():
    sample = torch.randn(64, 10).to(device)
    sample = model.decode(sample).cpu()
    plt.figure()
    show(make_grid(sample.view(64, 1, 28, 28)))
    plt.show()

NameError: name 'show' is not defined

In [ ]:
import music_gen_audio as mga
notes, pitchnames, n_vocab, network_input, normalized_input = mga.retrieve_notes_trained_data()
model = mga.create_network(normalized_input, n_vocab)

# input size is 1xlen(network_input)
# 1 x 58920
# Example: [311, 299, 313, 297, 299, 280, 299, 318, 313, 320, 143, 21, 45, 116, 49, 354, 277, 354, 277, 262, 277, 354, 277, 354, 277, 262, 277, 354, 277, 354, 277, 262, 277, 354, 277, 354, 277, 262, 277, 357, 354, 354, 320, 317, 342, 345, 354, 354, 333, 317, 342, 320, 354, 354, 4, 145, 320, 354, 354, 317, 342, 357, 311, 311, 345, 354, 290, 293, 311, 311, 314, 354, 290, 293, 311, 311, 229, 241, 357, 311, 311, 354, 290, 357, 354, 354, 320, 317, 342, 345, 354, 354, 333, 317, 342, 320, 354, 354, 4, 145]
rand_index = numpy.random.randint(0, len(network_input)-1)
init_input = network_input[rand_index]

# predict output from this sequence
prediction_output = mga.generate_notes(model, init_input, pitchnames, n_vocab)

mga.create_midi(prediction_output)